In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Assuming you have loaded your datasets into Pandas DataFrames named `patients` and `noteevents`
# You can read CSV files using pd.read_csv or other appropriate methods.

# Create a DataFrame for mimic.mimic3.patients
patients = pd.read_csv('/content/drive/My Drive/PATIENTS.csv')  # Replace 'path/to/patients.csv' with the actual path

# Create a DataFrame for mimic.mimic3.noteevents
noteevents = pd.read_csv('/content/drive/My Drive/NOTEEVENTS.csv')  # Replace 'path/to/noteevents.csv' with the actual path

In [ ]:
noteevents = noteevents.sort_values(by = 'CHARTDATE')

In [ ]:
# Step 1: Filter cases where the category is "discharge summary"
discharge_df = noteevents[noteevents['CATEGORY'] == 'Discharge summary']

# Step 2: Convert 'Encounter_Date' to datetime if not already
discharge_df['CHARTDATE'] = pd.to_datetime(discharge_df['CHARTDATE'])

# step 3: Add "within_date" column with offset set to 30 days from discharge date
discharge_df['within_date'] = discharge_df['CHARTDATE'] + pd.DateOffset(days=30)


In [ ]:
df_sorted = discharge_df.sort_values(by='SUBJECT_ID')

# Step 4: Keep only the first occurrence for each unique 'subject_id'
df_first_occurrence = df_sorted.drop_duplicates(subset='SUBJECT_ID', keep='first')

In [ ]:
#Combine discharge summary and noteevents data
merged_df = pd.merge(noteevents, df_first_occurrence, on='SUBJECT_ID', suffixes=('', '_first'))

# Step 5: Filter based on the condition that 'CHART DATE' is less than 'within_date' but more than the first discharge date
merged_filtered_df = merged_df[(merged_df['CHARTDATE_first']<merged_df['CHARTDATE']) & (merged_df['CHARTDATE']< merged_df['within_date'])]

In [ ]:
##### sort by chart time maybe
merged_filtered_df= merged_filtered_df.sort_values(by='CHARTTIME')

###and only keep the first one...
final_df = merged_filtered_df.drop_duplicates(subset='SUBJECT_ID', keep='first')

#merge with patients data
merged_patients_notes = patients.merge(final_df, on='SUBJECT_ID')

#load admissions data
admissions = pd.read_csv('/content/drive/My Drive/ADMISSIONS.csv')

#merge with demographic data from admissions.csv
admissions_first = admissions.drop_duplicates(subset='SUBJECT_ID', keep='first')
notes_demog = admissions_first.merge(merged_patients_notes, on='SUBJECT_ID')

#drop unecessary columns
notes_demog = notes_demog.drop(columns = ["ROW_ID","ADMITTIME", "DISCHTIME", "DOB", "DEATHTIME", "ADMISSION_TYPE","ADMISSION_LOCATION", "DISCHARGE_LOCATION", "EDREGTIME", "EDOUTTIME", "DIAGNOSIS", "HOSPITAL_EXPIRE_FLAG", "HAS_CHARTEVENTS_DATA", "ROW_ID_x", "ROW_ID_y", "DOD", "DOD_HOSP", "DOD_SSN", "EXPIRE_FLAG", "HADM_ID_x", "HADM_ID_y", "STORETIME", "DESCRIPTION", "CGID", "ISERROR", "ROW_ID_first","HADM_ID_first", "STORETIME_first", "DESCRIPTION_first", "CGID_first", "ISERROR_first", ], axis = 1)

#rename TEXT columns
notes_demog = notes_demog.rename(columns = {"TEXT": "Report2", "TEXT_first": "Report1"})